In [2]:
%load_ext cython

## Ref count issue

The below is not working, why?

The Cython will produce the error message Storing **unsafe C derivative of temporary  Python reference**. The reason is that concatenating the two Python strings produces a new Python string object that is referenced only by a temporary internal variable that Cython generates. As soon as the statement has finished, the temporary variable will be **decrefed** and the Python string deallocated, leaving s dangling. Since this code could not possibly work, Cython refuses to compile it.

In [8]:
%%cython
b1 = b'aa'
b2 = b'bb'
cdef char *buf = b1 + b2


Error compiling Cython file:
------------------------------------------------------------
...
b1 = b'aa'
b2 = b'bb'
cdef char *buf = b1 + b2
    ^
------------------------------------------------------------

C:\Users\Administrator\.ipython\cython\_cython_magic_c1dc3259b0dfff20ffcb605b57035318.pyx:3:5: Storing unsafe C derivative of temporary Python reference


Solution is simple

In [9]:
%%cython
b1 = b'aa'
b2 = b'bb'
p = b1 + b2
cdef char *buf = p

Note: why python doesn't have this issue? I think the reason is how python interpret "=", when we do

In [5]:
b1='aa'
b2='bb'
buf=b1+b2

we are actually, telling Python,ok, now, we have a temp from b1 + b2, then buf is a **alias** of temp, so the count of ref is still 1, after delete of temp.

However, Cython, on the other side, is making a copy of pointer, and this pointer is different from temp, meaning, buf and temp are pointers, and themselves also are stored somewhere, and they have different addresses.

# Why define cdef?

In [19]:
%%cython
def typed_fact(long n):
    if n<=1:
        return 1
    return n * typed_fact(n-1)

In [20]:
#%%cython
def typed_fact2(n):
    if n<=1:
        return 1
    return n * typed_fact2(n-1)

In [21]:
%timeit typed_fact(10)

242 ns ± 5.12 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [22]:
%timeit typed_fact2(10)

673 ns ± 6.96 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Few points:
- cdef must be in the same cell of the caller
- cdef, if omit the return type, then it's object
- cdef is not recommended to called from outside of Cython module, thus it's usually used as fast auxiliary functions to help def functions do their job.
- note, overflow may happen, when n > 21, this is wrong

In [43]:
%%cython
cimport cython
#from timeit import timeit
import time
cdef long c_fact(long n):
    if n<=1:
        return 1
    return n * c_fact(n-1)
# TODO: how to test it now!! Cannot use c_fact, so i had to write time..
# Also, calling timeit('c_fact(10)') will not work!
# Indirection!!!

#print(timeit("c_fact(10)"))
start = time.time()
for i in range(0, 1000000, 1):
    c_fact(10) # Note: this cell has %%cython, and calling c_fact must be inside this cell
end = time.time()
print((end - start)/1000000 * 1e9, 'ns')
# if you define a Python function, e.g., def --> cdef, it's fine, you can call it from a separate cell
#%timeit(c_fact(100))

12.956619262695312 ns


In [44]:
# Thus, a walkaround is wrap it!!!

In [51]:
%%cython

cdef long c_fact(long n):
    if n<=1:
        return 1
    return n * c_fact(n-1)


def wrap_c_fact(n):
    return c_fact

def lala(n):
    return 1

In [50]:
%timeit wrap_c_fact(10)

48.9 ns ± 0.881 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [52]:
%timeit lala(10)

25.6 ns ± 1.01 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


# Overloading

How? Use Fused Type, page 59